# Data/Package Import
Here the validation dataset is used in place of the whole dataset. When running on DIAS servers, we'll start with Tadhg's pipeline as if we were taking in everything from scratch. There will be 4 outputs from the script: A parquet file containing all the processed data and 3 parquet files ready for training, testing and validating our RNN. 

In [1]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
import datetime
import random
from tqdm.notebook import tqdm
from matplotlib.dates import DateFormatter
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import tensorflow as tf
from scipy import stats
from sklearn.preprocessing import RobustScaler
import seaborn as sns
import tensorflow.keras as keras
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


In [2]:
np.random.seed(51225)

In [3]:
data_directory = Path("../data")
crossing_list_csv = Path("mercury_boundary_crossing_list.csv")
validation_list_file = Path("validation_list.parquet")

In [4]:

crossing_list = pd.read_csv(
    data_directory / crossing_list_csv,
    sep=',', skipinitialspace=True, index_col=False,
    usecols=["Start_Date", "End_Date"],
    parse_dates=["Start_Date", "End_Date"], infer_datetime_format=True
)

# Converting transitions and Messenger data to datetime format
stopgap_minutes_delta = datetime.timedelta(minutes=10)

crossing_list["Start_Date"] -= stopgap_minutes_delta
crossing_list["End_Date"] += stopgap_minutes_delta

print("Importing Messenger Data...")

df = pd.read_parquet(data_directory / validation_list_file)

Importing Messenger Data...


In [5]:
crossing_list

,Start_Date,End_Date
0,2011-03-24 01:00:01,2011-03-24 01:23:35
1,2011-03-24 13:12:55,2011-03-24 13:33:17
2,2011-03-25 01:09:05,2011-03-25 01:31:49
3,2011-03-25 13:17:39,2011-03-25 13:40:03
4,2011-03-26 01:22:39,2011-03-26 01:45:29
...,...,...
11386,2014-03-16 12:08:00,2014-03-16 13:37:28
11387,2014-03-16 20:23:07,2014-03-16 21:05:03
11388,2014-03-17 03:51:22,2014-03-17 04:34:46
11389,2014-03-17 12:37:44,2014-03-17 13:45:04


In [6]:
transformer = RobustScaler().fit(df["B_X_MSO"].to_numpy().reshape(-1, 1))
df["B_X_MSO"] = transformer.transform(df["B_X_MSO"].to_numpy().reshape(-1, 1))

transformer = RobustScaler().fit(df["B_Y_MSO"].to_numpy().reshape(-1, 1))
df["B_Y_MSO"] = transformer.transform(df["B_Y_MSO"].to_numpy().reshape(-1, 1))

transformer = RobustScaler().fit(df["B_Z_MSO"].to_numpy().reshape(-1, 1))
df["B_Z_MSO"] = transformer.transform(df["B_Z_MSO"].to_numpy().reshape(-1, 1))
df = df[df.index < "March 2014"]

In [7]:
#Tadhg's preprocessing stuff, not needed for Jupyter 
"""start_date = datetime.date(2011,6,1)
strdate = datetime.datetime(2012,6,9,0,0,0)
enddate = datetime.datetime(2012,6,13,0,0,0)

#possible issues between 01/06/12-15/06/12

Filename = "Messenger_"+strdate.strftime("%Y%j")+".csv"
CNames = ['Year', 'DOY', 'Hour', 'Minute', 'Second', 'X_MSO', 'Y_MSO', 'Z_MSO', 'B_X_MSO', 'B_Y_MSO', 'B_Z_MSO']

df1 = pd.read_csv(dir_path + "/MESSENGER/"+Filename, sep = ' ', skipinitialspace = True, header = None, names = CNames)


#====Data assimilation loop

n_days = ((enddate -strdate).days)

for day in range(1,n_days):
    next_day = strdate+datetime.timedelta(days = day)
    
    next_day_str = next_day.strftime("%Y")+next_day.strftime("%j")
    
    Filename = "Messenger_"+next_day_str+".csv"
    df2 = pd.read_csv(dir_path + "/MESSENGER/"+Filename, sep = ' ', skipinitialspace = True, header = None, names = CNames)
    
    df = pd.concat([Data,Data2], ignore_index = True)

#====end loop
del n_days
del next_day
del next_day_str
del Data2

df["Year"] = df["Year"].astype(str).str.zfill(4)
df["DOY"] = df["DOY"].astype(str).str.zfill(3)
df["Hour"] = df["Hour"].astype(str).str.zfill(2)
df["Minute"] = df["Minute"].astype(str).str.zfill(2)
df["Second"] = pd.to_numeric(Data["Second"], errors = 'coerce').fillna(0).astype(int).astype(str).str.zfill(2)


df["Date"] = df["Year"] + df["DOY"] +df["Hour"] +df["Minute"] +df["Second"]

df["Date"] = pd.to_datetime(df["Date"],format = "%Y%j%H%M%S", errors = 'coerce')

print("Datetimes Created")

df.dropna(subset = ["Date"], inplace=True)

df = df.set_index("Date")
df = df.resample("1S").mean().interpolate()

p = np.degrees(np.arctan2(Data["Y_MSO"], df["X_MSO"]))+180.
dp = np.array(p[1:])- np.array(p[:-1])
orbitStartIndex = np.where(dp  < -180)[0]
lenp = len(p)

del p
del dp
    
loc = np.zeros(df.shape[0]).astype(int)

xxx = (crosssort["S_Date"] - df.index[0])
xxx = xxx.dt.total_seconds()

print("Created Location Array")

if np.nanmin(xxx) < 0:
    i = np.argmax(np.array(xxx.iloc[np.where(xxx < 0)]))
    if crosssort["Type"].iloc[i] == "BSI":
        loc[:] = 1
    
    if crosssort["Type"].iloc[i] == "MPO":
        loc[:] = 1
    
    if crosssort["Type"].iloc[i] == "MPI":
        loc[:] = 0
    
    if crosssort["Type"].iloc[i] == "BSO":
        loc[:] = 2

else:
    loc[:] = 2

del xxx

for i in range(len(crosssort)):
    index = np.where((Data.index - crosssort["S_Date"].iloc[i]).total_seconds() == 0)[0]
    if len(index) == 1:
        index = index[0]
        if crosssort["Type"].iloc[i] == "BSI":
            loc[index:] = -2
            
        if crosssort["Type"].iloc[i] == "MPO":
            loc[index:] = -1
            
        if crosssort["Type"].iloc[i] == "MPI":
            loc[index:] = -1
            
        if crosssort["Type"].iloc[i] == "BSO":
            loc[index:] = -2

    index = np.where((Data.index - crosssort["E_Date"].iloc[i]).total_seconds() == 0)[0]
    if len(index) == 1:
        index = index[0]
        if crosssort["Type"].iloc[i] == "BSI":
            loc[index:] = 1
            
        if crosssort["Type"].iloc[i] == "MPO":
            loc[index:] = 1
            
        if crosssort["Type"].iloc[i] == "MPI":
            loc[index:] = 0
            
        if crosssort["Type"].iloc[i] == "BSO":
            loc[index:] = 2

del crosssort
print("Filled Location Array")
df["Location"] = loc.tolist()

orbit = np.zeros(lenp)
for i in range(len(orbitStartIndex)):
    orbit[orbitStartIndex[i]:] = i

del lenp
del orbitStartIndex

df["OrbitNo"] = orbit.tolist()
print("Filled Orbit List")
"""



'start_date = datetime.date(2011,6,1)\nstrdate = datetime.datetime(2012,6,9,0,0,0)\nenddate = datetime.datetime(2012,6,13,0,0,0)\n\n#possible issues between 01/06/12-15/06/12\n\nFilename = "Messenger_"+strdate.strftime("%Y%j")+".csv"\nCNames = [\'Year\', \'DOY\', \'Hour\', \'Minute\', \'Second\', \'X_MSO\', \'Y_MSO\', \'Z_MSO\', \'B_X_MSO\', \'B_Y_MSO\', \'B_Z_MSO\']\n\ndf1 = pd.read_csv(dir_path + "/MESSENGER/"+Filename, sep = \' \', skipinitialspace = True, header = None, names = CNames)\n\n\n#====Data assimilation loop\n\nn_days = ((enddate -strdate).days)\n\nfor day in range(1,n_days):\n    next_day = strdate+datetime.timedelta(days = day)\n    \n    next_day_str = next_day.strftime("%Y")+next_day.strftime("%j")\n    \n    Filename = "Messenger_"+next_day_str+".csv"\n    df2 = pd.read_csv(dir_path + "/MESSENGER/"+Filename, sep = \' \', skipinitialspace = True, header = None, names = CNames)\n    \n    df = pd.concat([Data,Data2], ignore_index = True)\n\n#====end loop\ndel n_days\nd

# Labeling Transition Regions


In [8]:
stopgap_minutes_delta = datetime.timedelta(minutes=10)

crossing_list["Start_Date"] -= stopgap_minutes_delta
crossing_list["End_Date"] += stopgap_minutes_delta



In [9]:
# Initial column for transitions (to be changed later)
df["Transition"] = False


In [10]:

tqdm().pandas()


# Labeling date ranges where transitions occurred (with a 10 minute buffer either side)

print("Labeling Data...")

start_time = datetime.datetime.now()
for idx, row in tqdm(crossing_list.iterrows(), total=len(crossing_list), position = 0, leave = True):
    df.loc[row.Start_Date:row.End_Date, "Transition"] = True

del crossing_list

print("Elapsed:", datetime.datetime.now() - start_time)
print(df.Transition.value_counts())


0it [00:00, ?it/s]

Labeling Data...


  0%|          | 0/11391 [00:00<?, ?it/s]

Elapsed: 0:01:53.253003
False    14222056
True      5913940
Name: Transition, dtype: int64


In [11]:
batch = 0
x = 0
total_data_size = len(df)
df["Batch"] = -1

pbar = tqdm(total=len(df) , position = 0, leave = True)

while x < total_data_size-300:
    if df["Transition"].iloc[x-150] == False and df["Transition"].iloc[x+150] == False:
        df.iloc[x-150:x+150]["Batch"] = batch
        batch +=1
        x += 300
        pbar.update(300)
    else: 
        x += 1
        pbar.update(1)
        pass

  0%|          | 0/20135996 [00:00<?, ?it/s]

<ipython-input-11-7fe27720e3a1>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[x-150:x+150]["Batch"] = batch


In [12]:
"""#Sanity Check
bad_sequences = [] 
df_1 = df[["Location", "Batch"]]
for b in tqdm(range(0,batch), total=batch, position = 0, leave = True): 
    if len(df_1.loc[df_1["Batch"]== b]["Location"].unique()) != 1:
        bad_sequences.append(b)"""

'#Sanity Check\nbad_sequences = [] \ndf_1 = df[["Location", "Batch"]]\nfor b in tqdm(range(0,batch), total=batch, position = 0, leave = True): \n    if len(df_1.loc[df_1["Batch"]== b]["Location"].unique()) != 1:\n        bad_sequences.append(b)'

# Creating Testing/Training/Validation Split

In [13]:
train_features = np.empty(shape = [int(batch*0.6), 300, 3])
test_features = np.empty(shape = [int(batch*0.2), 300, 3])
validation_features = np.empty(shape = [int(batch*0.2), 300, 3])

train_targets = np.empty(shape = [int(batch*0.6)])
test_targets = np.empty(shape = [int(batch*0.2)])
validation_targets = np.empty(shape = [int(batch*0.2)])

train_len = len(train_targets)
test_len = len(test_targets)

In [14]:
df_features = df[["B_X_MSO", "B_Y_MSO", "B_Z_MSO"]]
df_targets = df["Location"]
df_batches = df["Batch"]

In [15]:
train_index = 0
test_index = 0
validation_index = 0

pbar = tqdm(total=batch , position = 0, leave = True)
b = 1
while b < batch -1: 
    dataset_det = np.random.uniform(0,1)
    if dataset_det < 0.6 and train_index < train_len: 
        train_features[train_index] = df_features.loc[df_batches == b]
        train_targets[train_index] = df_targets.loc[df_batches == b].unique()[0]
        train_index +=1
        b += 1
        pbar.update(1)
    elif dataset_det < 0.8 and test_index < test_len: 
        test_features[test_index] = df_features.loc[df_batches == b]
        test_targets[test_index] = df_targets.loc[df_batches == b].unique()[0]
        test_index +=1
        b += 1
        pbar.update(1)
    elif validation_index < test_len:
        validation_features[validation_index] = df_features.loc[df_batches == b]
        validation_targets[validation_index] = df_targets.loc[df_batches == b].unique()[0]        
        validation_index +=1
        b +=1
        pbar.update(1)

  0%|          | 0/46543 [00:00<?, ?it/s]

In [16]:
train_features

array([[[  8.24728985,   4.09839609, -18.5643198 ],
        [  8.26497245,   4.12083351, -18.52083222],
        [  8.28031515,   4.1444382 , -18.4784707 ],
        ...,
        [  8.16429714,   7.37711297,  -1.39090568],
        [  8.1470292 ,   7.36842333,  -1.34967022],
        [  8.13393756,   7.36470537,  -1.27717304]],

       [[  8.12016468,   7.34819074,  -1.22537401],
        [  8.10206741,   7.34421339,  -1.17470106],
        [  8.08334814,   7.33660456,  -1.12027455],
        ...,
        [  2.49380961,   2.61782889,   6.71998499],
        [  2.48302826,   2.59768276,   6.73516006],
        [  2.47286891,   2.57892006,   6.75108585]],

       [[  0.61050886,  -0.32030608,   0.35052818],
        [  0.61376696,  -0.31719338,   0.33170679],
        [  0.61743973,  -0.30361852,   0.33696177],
        ...,
        [  0.637403  ,  -0.18542216,   0.13898869],
        [  0.63654404,  -0.18343349,   0.1723953 ],
        [  0.64226053,  -0.18460075,   0.17400397]],

       ...,

      

In [17]:
validation_features

array([[[ 2.46125822e+00,  2.55803900e+00,  6.76272186e+00],
        [ 2.44677448e+00,  2.53499633e+00,  6.76626093e+00],
        [ 2.43069131e+00,  2.51606070e+00,  6.76636817e+00],
        ...,
        [-3.17161306e-01, -3.21257187e-01,  4.91999571e+00],
        [-3.28238848e-01, -3.10276253e-01,  4.90868143e+00],
        [-3.41063918e-01, -2.99252086e-01,  4.88610649e+00]],

       [[ 7.33872401e-01, -2.63672128e-01,  1.44243659e-01],
        [ 7.36656596e-01, -2.47200726e-01,  1.51750764e-01],
        [ 7.34138973e-01, -2.51048377e-01,  1.31535203e-01],
        ...,
        [ 7.25519815e-01, -3.52297782e-01,  6.53118130e-02],
        [ 7.23772288e-01, -3.64705374e-01,  6.27379484e-02],
        [ 7.20810379e-01, -3.58566426e-01,  4.55252292e-02]],

       [[ 7.43617084e-01, -3.19398210e-01,  1.02686471e-01],
        [ 7.41691843e-01, -3.17755393e-01,  1.09013888e-01],
        [ 7.34701736e-01, -3.24931910e-01,  1.05099469e-01],
        ...,
        [ 7.54398436e-01, -3.39284942e-01,

In [18]:
test_features

array([[[ 0.70819264, -0.38398686,  0.04305861],
        [ 0.72084   , -0.35748563,  0.06230897],
        [ 0.71974409, -0.36773162,  0.05539171],
        ...,
        [ 0.74415023, -0.33876616,  0.06016408],
        [ 0.73902612, -0.33124379,  0.07319427],
        [ 0.74826728, -0.33305953,  0.06729583]],

       [[ 0.99742314, -0.16497341,  0.23239852],
        [ 0.99843019, -0.17353335,  0.22682181],
        [ 0.99715657, -0.1718473 ,  0.21872486],
        ...,
        [ 1.00882649, -0.28468289,  0.2874685 ],
        [ 1.01158107, -0.27145389,  0.2850555 ],
        [ 1.01368402, -0.27141066,  0.2678964 ]],

       [[ 1.02748652, -0.15386278,  0.24210413],
        [ 1.02479119, -0.1433574 ,  0.24623304],
        [ 1.02452461, -0.11914746,  0.21245107],
        ...,
        [ 1.02144423, -0.17491678,  0.1929326 ],
        [ 1.01483917, -0.17915352,  0.19539922],
        [ 1.01593507, -0.19022092,  0.17941981]],

       ...,

       [[ 0.34316687, -0.20232588, -0.16874899],
        [ 0

# Recurrent Neural Network
This section will not be included in the final preprocessing script. It is just here to show how it will be fed into the RNN and to prototype some plots

In [19]:
def build_model():
    lstm_layer = keras.layers.RNN(
        keras.layers.LSTMCell(units), input_shape=(300, 3)
    )
    model = keras.models.Sequential(
        [
            lstm_layer,
            keras.layers.Dropout(0.2),
            keras.layers.Dense(output_size),
        ]
    )
    return model

In [20]:
batch_size = 64
units = 64
output_size = 3
training_sample_sizes = 0.05

In [21]:
model = build_model()

opt = tf.keras.optimizers.Adam(learning_rate = 1e-3, decay = 1e-5)

In [22]:
checkpoint_path = "training_3/cp.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=opt,
    metrics=["accuracy"],
)

In [23]:

weights = np.unique(train_targets, return_counts = True)[1]/len(train_targets)

In [24]:
class_weight = {0:weights[0] ,1:weights[1] , 2:weights[2]}

In [25]:
model.fit(train_features, train_targets,  
        batch_size=batch_size, 
        validation_data = (validation_features, validation_targets),
        epochs=10,
        callbacks=[cp_callback],
         class_weight = class_weight)


Epoch 1/10
437/437 [==============================] - 59s 134ms/step - loss: 0.1845 - accuracy: 0.8035 - val_loss: 0.4069 - val_accuracy: 0.8706

Epoch 00001: saving model to training_3/cp.ckpt
Epoch 2/10
437/437 [==============================] - 55s 125ms/step - loss: 0.0445 - accuracy: 0.8850 - val_loss: 0.4016 - val_accuracy: 0.8951

Epoch 00002: saving model to training_3/cp.ckpt
Epoch 3/10
437/437 [==============================] - 56s 128ms/step - loss: 0.0418 - accuracy: 0.8918 - val_loss: 0.4031 - val_accuracy: 0.8975

Epoch 00003: saving model to training_3/cp.ckpt
Epoch 4/10
437/437 [==============================] - 55s 127ms/step - loss: 0.0424 - accuracy: 0.8912 - val_loss: 0.4454 - val_accuracy: 0.8832

Epoch 00004: saving model to training_3/cp.ckpt
Epoch 5/10
437/437 [==============================] - 55s 127ms/step - loss: 0.0402 - accuracy: 0.8901 - val_loss: 0.3910 - val_accuracy: 0.8927

Epoch 00005: saving model to training_3/cp.ckpt
Epoch 6/10
437/437 [==========

# Plots and RNN Analysis
Again, just here for prototyping reasons. This is a work in progress so if you hit run all, I've put a cell here which will error so it will go: 

- Data Import 
- Transition Removal
- Sequencing
- Test/Train/Val split
- Error 
- Plots and analysis 
- Outputting files

In [26]:
error here 

SyntaxError: invalid syntax (<ipython-input-26-fc9b6cdd1f7a>, line 1)

In [ ]:
#df.iloc[x-2000:x+2000]["Location"].replace(mapping).value_counts()

In [ ]:
#red orange purple  yellow blue 
mapping = {"0": "#b80000", "-1":"#5300eb", "1":"#1273de", "-2":"#e65100", "2":"#fccb00"}

## Create custom lines
r_line = plt.Line2D((0,1),(0,0), color='#b80000')
p_line = plt.Line2D((0,1),(0,0), color='#5300eb')
b_line = plt.Line2D((0,1),(0,0), color='#1273de')
o_line = plt.Line2D((0,1),(0,0), color='#e65100')
y_line = plt.Line2D((0,1),(0,0), color='#fccb00')


# Create plot
fig, ax = plt.subplots()

def gen_repeating(s):
    i = 0
    while i < len(s):
        j = i
        while j < len(s) and s[j] == s[i]:
            j += 1
        yield (s[i], i, j-1)
        i = j
        


## Add px_last lines
for color, start, end in gen_repeating(df.iloc[x-2000:x+2000]["Location"].replace(mapping)):
    if start > 0: # make sure lines connect
        start -= 1
    idx = df.iloc[x-2000:x+2000].index[start:end+1]
    ax.semilogy(np.sqrt(df.loc[idx, 'B_X_MSO']**2+
                   df.loc[idx, 'B_Y_MSO']**2+
                   df.loc[idx, 'B_Z_MSO']**2),
            color=color,
            label='',
            linewidth = 2)

handles, labels = ax.get_legend_handles_labels()


## Create legend from custom artist/label lists
ax.legend(
    handles + [r_line, 
               p_line, 
               b_line, 
               o_line,
               y_line 
               ],
    labels + [
        'Magnetosphere',
        "Magnetopause",
        'Magnetosheath',
        "Bow Shock",       
        'Solar Wind'
    ],
    fontsize=20,
    loc='upper right',
    bbox_to_anchor=(1.2, 1.0175)
)

#Formating x-axis dates
ax.xaxis_date()
fig.autofmt_xdate()
formatter =  DateFormatter('%Y-%h-%d %H:%M')
ax.xaxis.set_major_formatter(formatter)

plt.ylabel("|B| (nT)")
# Display plot
plt.show()

In [ ]:
# Outputing Parquet Files

In [ ]:
# Declare paths to the data directory and files of interest
data_directory = Path("../data")
data_files = [
    "test_list.csv",
    "train_list.csv",
    "validation_list.csv"
]

print("Parsing data files...")
start_time = datetime.datetime.now()

for data_file in data_files:
    # Iterate over each file,
    print(f"Parsing {data_file}...")
    start_time_file = datetime.datetime.now()

    # We define the data types to speed up reading and ensure they're saved properly in the Parquet
    df = pd.read_csv(
        data_directory / data_file,
        sep=',', skipinitialspace=True, index_col="Date",
        dtype={
          'X_MSO': np.float64, 'Y_MSO': np.float64, 'Z_MSO': np.float64,
          'B_X_MSO': np.float64, 'B_Y_MSO': np.float64, 'B_Z_MSO': np.float64,
          'Location': 'category',
          'OrbitNo': np.int32
        },
        parse_dates=["Date"], infer_datetime_format=True
    )

    # Now we save to Parquet, and return the space saving (a bit unnecessary...)
    output_file = data_file.split('.')[0]+'.parquet'
    df.to_parquet(data_directory / output_file)
    print(
        f"Elapsed {datetime.datetime.now() - start_time_file}, size from "
        f"{(data_directory/data_file).stat().st_size / 1024**3:.2f}GB to "
        f"{(data_directory / output_file).stat().st_size / 1024**3:.2f}GB"
    )

print('Total elapsed:', datetime.datetime.now() - start_time)

Data.to_csv(dir_path+"/MESSENGER_data.csv")
del Data
del orbit